In [1]:
#import libraries
import pandas as pd
import numpy as np
import openpyxl as op
from google.colab import files
from openpyxl.utils.dataframe import dataframe_to_rows

#create upload link
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
#copy json file in a directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#permission to access kaggle data
!chmod 600 ~/.kaggle/kaggle.json

#Download kaggle data
!kaggle datasets download -d wendykan/lending-club-loan-data

#Unzip kaggle data
from zipfile import ZipFile
file_name = "lending-club-loan-data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Unzip Done')

#import kaggle Data
data = pd.read_csv('loan.csv')
total_rows_Data=len(data.axes[0])
print('Total Rows in Data:', total_rows_Data)

 98% 707M/720M [00:06<00:00, 135MB/s]
100% 720M/720M [00:06<00:00, 117MB/s]
Unzip Done


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total Rows in Data: 2260668


In [3]:
#Create Working Dataframe
df = data.loc[:, ['annual_inc', 'loan_amnt', 'annual_inc_joint', 'issue_d', 'out_prncp', 'grade', 'loan_status', 'dti', 'dti_joint', 'acc_now_delinq', 'inq_last_12m', 'inq_fi', 'hardship_flag', 'collections_12_mths_ex_med']]
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

Total Rows in DataFrame 2260668


In [4]:
#Replacing NaN value with 0
df['annual_inc'] = df['annual_inc'].replace(np.nan, 1)
df['annual_inc'] = df['annual_inc'].replace(0, 1)
df['annual_inc_joint'] = df['annual_inc_joint'].replace(np.nan, 0)
df['dti'] = df['dti'].replace(np.nan, 400000000)
df['dti_joint'] = df['dti_joint'].replace(np.nan, 0)
df['inq_last_12m'] = df['inq_last_12m'].replace(np.nan, 1000)
df['inq_fi'] = df['inq_fi'].replace(np.nan, 1000)


#Finding Total number of rows after data cleaning
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

#Calculate Total income, dti and inq
sumColumn = df['annual_inc'] + df['annual_inc_joint']
df['total_inc'] = sumColumn

#Rounded real values to the nearest whole dollar
#df["total_inc"] = df["total_inc"].astype(int)

df['debt'] = df['dti'] * df['annual_inc']
df['debt_joint'] = df['dti_joint'] * df['annual_inc_joint']

df['total_dti'] = (df['debt'] + df['debt_joint']) / (df['annual_inc'] + df['annual_inc_joint'])

sumColumn2 = df['inq_last_12m'] + df['inq_fi']
df['total_inq'] = sumColumn2

#Generate Bands according to total_dti
dti_band = [-2, 10, 15, 20, 25, 100, 65000]
dti_bands_name =[' <=10', ' >10-15', ' >15-20', ' >20-25', ' >25', 'Missing']
df['dti_band'] = pd.cut(df['total_dti'], dti_band, labels = dti_bands_name)

#Generate Bands according to total_inquiry
inq_band = [-1, 0, 1, 2, 4, 100, 25000]
inq_bands_name =[' <=0', ' >0-1', ' >1-2', ' >2-4', ' >4', 'Missing']  
df['inq_band'] = pd.cut(df['total_inq'], inq_band, labels = inq_bands_name)

#Generate Band according to Total income
inc_bands = [-1, 1, 45000, 60000, 80000, 110000, 1100000000]
inc_bands_name =["Missing", "0-45K", ">45K-60K", ">60K-80K", ">80K-110K", ">110K"]
df['inc_band'] = pd.cut(df['total_inc'], inc_bands, labels = inc_bands_name)

#Removing accounts with Missing Value in DTI and Income
df.drop(df[df['dti_band'] == 'Missing'].index, inplace=True)
df.drop(df[df['inc_band'] == 'Missing'].index, inplace=True)

total_rows = len(df.axes[0])
print('Removed Segments with Missing Values of DTI and Income',total_rows)

#Removing Segments with DTI greater than 20
df.drop(df[df['dti_band'] == ' >20-25'].index, inplace=True)
df.drop(df[df['dti_band'] == ' >25'].index, inplace=True)

total_rows = len(df.axes[0])
print('Removed Segments which have DTI greater than 20:',total_rows)

#Removing Segments with Inquiry greater than 4
df.drop(df[df['inq_band'] == ' >4'].index, inplace=True)

total_rows = len(df.axes[0])
print('Removed Segments which have Inquiry Greater than 4:',total_rows)

#Removing Segments with Income less than 80K
df.drop(df[df['inc_band'] == '0-45K'].index, inplace=True)
df.drop(df[df['inc_band'] == '>45K-60K'].index, inplace=True)
df.drop(df[df['inc_band'] == '>60K-80K'].index, inplace=True)

total_rows = len(df.axes[0])
print('Removed Segments which have Income less than 80K:',total_rows)

#Removing Segments with Grade C, D, E, F, G
df.drop(df[df['grade'] == 'C'].index, inplace=True)
df.drop(df[df['grade'] == 'D'].index, inplace=True)
df.drop(df[df['grade'] == 'E'].index, inplace=True)
df.drop(df[df['grade'] == 'F'].index, inplace=True)
df.drop(df[df['grade'] == 'G'].index, inplace=True)

total_rows = len(df.axes[0])
print('Removed Segments which have Grade C, D, E, F, G:',total_rows)



Total Rows in DataFrame 2260668
Removed Segments with Missing Values of DTI and Income 2258953
Removed Segments which have DTI greater than 20: 1342441
Removed Segments which have Inquiry Greater than 4: 1164597
Removed Segments which have Income less than 80K: 471916
Removed Segments which have Grade C, D, E, F, G: 283090


In [5]:
#Eliminating Closed Account
indexNames = df[(df['loan_status'] == 'Fully Paid')].index
df.drop(indexNames , inplace=True)

#Finding Total number of rows 
total_rows_prev = total_rows
total_rows = len(df.axes[0])
print('Eliminated Fully paid Account:', total_rows_prev - total_rows, total_rows)

#Eliminating Charged Off or Bad Account

#Assigning Flag to Bad account
df['bad_acc'] = df['loan_status']
df['bad_acc'].replace(to_replace = ['Charged Off', 'Current', 'Default',	'Does not meet the credit policy. Status:Charged Off',	'Does not meet the credit policy. Status:Fully Paid',	'Fully Paid',	'In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)'],
                           value = [1, 0, 0, 0, 0, 0, 0, 0, 0],
                         inplace = True)

indexNames = df[(df['bad_acc'] == 1)].index
df.drop(indexNames , inplace=True)

#Finding Total number of rows 
total_rows_prev = total_rows
total_rows = len(df.axes[0])
print('Eliminated Charged Off Accounts:', total_rows_prev - total_rows, total_rows)


#Create buraeu bad indicator
df['hardship_flag_bin'] = df['hardship_flag']
df['hardship_flag_bin'].replace(to_replace = ['Y', 'N'],
                                value = [1, 0],
                              inplace = True)

df['delinquency_flag'] = df['loan_status']
df['delinquency_flag'].replace(to_replace = ['Charged Off', 'Current', 'Default', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Fully Paid', 'In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)'],
                                value = [0, 0, 1, 0, 0, 0, 1, 1, 1],
                              inplace = True)

df['delinq_flag'] = df['acc_now_delinq']
df['delinq_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['collection_flag'] = df['collections_12_mths_ex_med']
df['collection_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['bureau_bad'] = df['hardship_flag_bin'] + df['delinquency_flag'] + df['collection_flag']

df['bureau_bad'].replace(to_replace = [0, 1, 2, 3],
                                value = [0, 1, 1, 1],
                              inplace = True)

#Eliminating Bureau Bad Account
indexNames = df[(df['bureau_bad'] == 1)].index
df.drop(indexNames , inplace=True)

#Finding Total number of rows 
total_rows_prev = total_rows
total_rows = len(df.axes[0])
print('Eliminated Bureau Bad Accounts:', total_rows_prev - total_rows, total_rows)

#Eliminating last six month data
indexNames = df[(df['issue_d'] == 'Jul-2018') | (df['issue_d'] == 'Aug-2018') | (df['issue_d'] == 'Sep-2018') | (df['issue_d'] == 'Oct-2018') | (df['issue_d'] == 'Nov-2018') | (df['issue_d'] == 'Dec-2018')].index
df.drop(indexNames , inplace=True)

#Finding Total number of rows 
total_rows_prev = total_rows
total_rows = len(df.axes[0])
print('Eliminated last 6 Month Data:', total_rows_prev - total_rows, total_rows)

#Eliminating Delinquent Account
indexNames = df[(df['delinq_flag'] == 1)].index
df.drop(indexNames , inplace=True)

#Finding Total number of rows 
total_rows_prev = total_rows
total_rows = len(df.axes[0])
print('Eliminated Delinquent Account:', total_rows_prev - total_rows, total_rows)

Eliminated Fully paid Account: 144236 138854
Eliminated Charged Off Accounts: 12418 126436
Eliminated Bureau Bad Accounts: 3895 122541
Eliminated last 6 Month Data: 35252 87289
Eliminated Delinquent Account: 313 86976


**Data Parsing**

In [6]:
#Replacing NaN value with 0
df['annual_inc'] = df['annual_inc'].replace(np.nan, 1)
df['annual_inc'] = df['annual_inc'].replace(0, 1)
df['annual_inc_joint'] = df['annual_inc_joint'].replace(np.nan, 0)
df['dti'] = df['dti'].replace(np.nan, 400000000)
df['dti_joint'] = df['dti_joint'].replace(np.nan, 0)
df['acc_now_delinq'] = df['acc_now_delinq'].replace(np.nan, 0)
df['inq_last_12m'] = df['inq_last_12m'].replace(np.nan, 1000)
df['inq_fi'] = df['inq_fi'].replace(np.nan, 1000)
df['collections_12_mths_ex_med'] = df['collections_12_mths_ex_med'].replace(np.nan, 0)

#Finding Total number of rows after data cleaning
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

#Calculate Total income, dti, inq and %pay_down
sumColumn = df['annual_inc'] + df['annual_inc_joint']
df['total_inc'] = sumColumn

#Rounded real values to the nearest whole dollar
#df["total_inc"] = df["total_inc"].astype(int)

df['debt'] = df['dti'] * df['annual_inc']
df['debt_joint'] = df['dti_joint'] * df['annual_inc_joint']

df['total_dti'] = (df['debt'] + df['debt_joint']) / (df['annual_inc'] + df['annual_inc_joint'])

sumColumn2 = df['inq_last_12m'] + df['inq_fi']
df['total_inq'] = sumColumn2

df['%pay_down'] = (1 - (df['out_prncp'] / df['loan_amnt'])) *100

#Generate Bands according to total_dti
dti_band = [-2, 10, 15, 20, 25, 100, 65000]
dti_bands_name =[' <=10', ' >10-15', ' >15-20', ' >20-25', ' >25', 'Missing']
df['dti_band'] = pd.cut(df['total_dti'], dti_band, labels = dti_bands_name)

#Generate Bands according to total_inquiry
inq_band = [-1, 0, 1, 2, 4, 100, 25000]
inq_bands_name =[' <=0', ' >0-1', ' >1-2', ' >2-4', ' >4', 'Missing']  
df['inq_band'] = pd.cut(df['total_inq'], inq_band, labels = inq_bands_name)

#Generate Band according to Total income
inc_bands = [-1, 1, 45000, 60000, 80000, 110000, 1100000000]
inc_bands_name =["Missing", "0-45K", ">45K-60K", ">60K-80K", ">80K-110K", ">110K"]
df['inc_band'] = pd.cut(df['total_inc'], inc_bands, labels = inc_bands_name)

#Generate Bands according to %pay down
pay_down_band = [-1, 20, 30, 40, 50, 70, 100]
pay_down_bands_name =['<=20', '>20-30', '>30-40', '>40-50', '>50-70', '>70']
df['%pay_down_band'] = pd.cut(df['%pay_down'], pay_down_band, labels = pay_down_bands_name)

#Create bad indicator
#IGP - Including Grace Period
#EGP - Excluding Grace Period
df['bad_flag_IGP'] = df['loan_status']
df['bad_flag_IGP'].replace(to_replace = ['Does not meet the credit policy. Status:Charged Off', 'Charged Off', 'In Grace Period',  'Default', 'Late (16-30 days)', 'Late (31-120 days)', 'Current', 'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'],
                                value = [1, 1, 1, 1, 1, 1, 0, 0, 0],
                              inplace = True)

df['bad_flag_EGP'] = df['loan_status']
df['bad_flag_EGP'].replace(to_replace = ['Does not meet the credit policy. Status:Charged Off', 'Charged Off', 'In Grace Period',  'Default', 'Late (16-30 days)', 'Late (31-120 days)', 'Current', 'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'],
                                value = [1, 1, 0, 1, 1, 1, 0, 0, 0],
                              inplace = True)

#Create buraeu bad indicator
df['hardship_flag_bin'] = df['hardship_flag']
df['hardship_flag_bin'].replace(to_replace = ['Y', 'N'],
                                value = [1, 0],
                              inplace = True)

df['delinq_flag'] = df['acc_now_delinq']
df['delinq_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['collection_flag'] = df['collections_12_mths_ex_med']
df['collection_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['bureau_bad'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag']

df['bureau_bad'].replace(to_replace = [0, 1, 2, 3],
                                value = [0, 1, 1, 1],
                              inplace = True)

#OA - Over All
df['OA_bad_acc_IGP'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag'] + df['bad_flag_IGP']

df['OA_bad_acc_IGP'].replace(to_replace = [0, 1, 2, 3, 4],
                            value = [0, 1, 1, 1, 1],
                              inplace = True)

#OA - Over All
df['OA_bad_acc_EGP'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag'] + df['bad_flag_EGP']

df['OA_bad_acc_EGP'].replace(to_replace = [0, 1, 2, 3, 4],
                            value = [0, 1, 1, 1, 1],
                              inplace = True)

#Temp column to calculate the total no of accounts in different segment
df['total_acc'] = df['hardship_flag']
df['total_acc'].replace(to_replace = ['Y', 'N'],
                        value = [1, 1],
                        inplace = True)

df['%pay_down'] = 1 - (df['out_prncp'] / df['loan_amnt'])

df.tail()

Total Rows in DataFrame 86976


,annual_inc,loan_amnt,annual_inc_joint,issue_d,out_prncp,grade,loan_status,dti,dti_joint,acc_now_delinq,inq_last_12m,inq_fi,hardship_flag,collections_12_mths_ex_med,total_inc,debt,debt_joint,total_dti,total_inq,dti_band,inq_band,inc_band,bad_acc,hardship_flag_bin,delinquency_flag,delinq_flag,collection_flag,bureau_bad,%pay_down,%pay_down_band,bad_flag_IGP,bad_flag_EGP,OA_bad_acc_IGP,OA_bad_acc_EGP,total_acc
2260445,170000.0,40000,0.0,Oct-2017,23817.93,B,Current,15.18,0.00,0.0,0.0,0.0,N,0.0,170000.0,2580600.0,0.0,15.18,0.0,>15-20,<=0,>110K,0,0,0,0.0,0.0,0.0,0.404552,>40-50,0,0,0.0,0.0,1
2260458,150000.0,17000,0.0,Oct-2017,9777.63,A,Current,15.33,0.00,0.0,0.0,0.0,N,0.0,150000.0,2299500.0,0.0,15.33,0.0,>15-20,<=0,>110K,0,0,0,0.0,0.0,0.0,0.424845,>40-50,0,0,0.0,0.0,1
2260490,125000.0,32000,0.0,Oct-2017,18884.53,B,Current,10.03,0.00,0.0,1.0,1.0,N,0.0,125000.0,1253750.0,0.0,10.03,2.0,>10-15,>1-2,>110K,0,0,0,0.0,0.0,0.0,0.409858,>40-50,0,0,0.0,0.0,1
2260540,150000.0,16500,0.0,Oct-2017,9737.21,B,Current,17.55,0.00,0.0,2.0,0.0,N,0.0,150000.0,2632500.0,0.0,17.55,2.0,>15-20,>1-2,>110K,0,0,0,0.0,0.0,0.0,0.409866,>40-50,0,0,0.0,0.0,1
2260649,40000.0,35000,240000.0,Oct-2017,19892.41,A,Current,66.19,11.03,0.0,4.0,0.0,N,0.0,280000.0,2647600.0,2647200.0,18.91,4.0,>15-20,>2-4,>110K,0,0,0,0.0,0.0,0.0,0.431645,>40-50,0,0,0.0,0.0,1


Summary according to %pay down

In [7]:
demographic = '%pay_down_band'

x = df.groupby(
   [demographic, 'OA_bad_acc_EGP']
).agg( 
    {
         'OA_bad_acc_EGP': "count",
    }
)

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False)

df1 = df1.pivot(index = demographic, columns = 'OA_bad_acc_EGP', values = 'OA_bad_acc_EGP.1')

df1 = df1.replace(np.nan, 0)
df1.head(10)

OA_bad_acc_EGP,0.0,1.0
%pay_down_band,,
<=20,8670,0
>20-30,18641,0
>30-40,13622,0
>40-50,11116,0
>50-70,18521,0
>70,16386,20


In [39]:
df['%pay_down'] = (1 - (df['out_prncp'] / df['loan_amnt'])) *100
#Generate Bands according to %pay down


values, count = np.unique(df['%pay_down_band'], return_counts=True)

dfTemp = pd.DataFrame()
dfTemp['Values'] = values
dfTemp['Count'] = count

dfTemp.head(10)

,Values,Count
0,<=20,8670
1,>20-30,18641
2,>30-40,13622
3,>40-50,11116
4,>50-70,18521
5,>70,16406
